In [2]:
# リスト 2.1.26
# SPARQL 検索例1
# 東証一部上場企業の一覧を取得し、名称と概要を表示する
# ------------------------------------------------------------------------------------------------
# 目的
#   - 日本語DBpedia（ja.dbpedia.org）のSPARQLエンドポイントに問い合わせ、
#     「Category:東証一部上場企業」に関連する企業の rdfs:label（名称）と dbo:abstract（概要）を取得する。
#
# 理論メモ（SPARQL/DBpediaの前提）
#   - SPARQLは RDF（三つ組: 主語-述語-目的語）に対する問い合わせ言語。
#   - DBpediaでは Wikipedia の構造化抽出を RDF として公開しており、語い（ボキャブラリ）に
#       * rdfs:label                … リソースの人間可読名（言語タグ付き：@ja, @en など）
#       * dbo:abstract              … 要約文（言語タグ付き）
#       * dcterms:subject           … 記事のカテゴリ（Category:... へのリンク）
#     などがある。
#   - 本クエリは wikiPageWikiLink を用いて「Category ページへリンクしているリソース」を拾っている。
#     ただし”カテゴリ所属”の正道は dcterms:subject を使うのが一般的（参考: ?company dcterms:subject <...Category:東証一部上場企業>）。
#     ここでは元コードの挙動を尊重し変更しない（改善案はコメントとして提示）。
#
# 注意点
#   - PREFIX 句を明示していないため、エンドポイント側で rdfs, dbo 等の既定プレフィックスが定義されていないと失敗する。
#     多くの DBpedia エンドポイントでは既定が入っていることが多いが、厳密には PREFIX を書くのが標準的。
#   - 言語タグのフィルタが無いので、多言語が混在する可能性がある（FILTER(lang(?name)='ja') などの導入が望ましい）。
#   - 東証の市場区分は再編（東証一部→プライム等）されているため、データの鮮度・カテゴリの存在は時期依存。
#   - 大量件数を返す可能性があるため、運用では LIMIT/OFFSET もしくは CONSTRUCT/ページングを検討。
# ------------------------------------------------------------------------------------------------

from SPARQLWrapper import SPARQLWrapper

# 日本語DBpediaのSPARQLエンドポイントに接続（returnFormat='json' は SPARQLWrapper の戻り形式指定）
sparql = SPARQLWrapper(endpoint="http://ja.dbpedia.org/sparql", returnFormat="json")

# SPARQLクエリを設定
# - distinct: 重複除去
# - パターン:
#     ?company dbo:wikiPageWikiLink Category:東証一部上場企業 .
#     ?company rdfs:label ?name .
#     ?company dbo:abstract ?abstract .
#   ※ 改善案（コメントのみ）:
#     1) PREFIX 宣言の明示:
#        PREFIX dbo: <http://dbpedia.org/ontology/>
#        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
#        PREFIX dct: <http://purl.org/dc/terms/>
#     2) カテゴリ所属の正道:
#        ?company dct:subject <http://ja.dbpedia.org/resource/Category:東証一部上場企業> .
#     3) 日本語限定（言語タグ）:
#        FILTER (lang(?name) = 'ja') .
#        FILTER (lang(?abstract) = 'ja') .
sparql.setQuery(
    """
select distinct ?name ?abstract where {
    ?company <http://dbpedia.org/ontology/wikiPageWikiLink> <http://ja.dbpedia.org/resource/Category:東証一部上場企業> .
    ?company rdfs:label ?name .
    ?company <http://dbpedia.org/ontology/abstract> ?abstract .
}
"""
)

# クエリ送信 → JSON に変換（.convert()）
results = sparql.query().convert()

# 検索結果から名称(name)と概要(abstract)を抽出
# - SPARQLの JSON 形式では、各バインディングは { "var": { "type": "...", "value": "...", "xml:lang": "ja" } } の形。
# - 改行を落として一行化している（出力見やすさのため）。NLP用途では段落境界を保持する方が望ましい場合もある。
import json

items = []
for result in results["results"]["bindings"]:
    item = {}
    # ラベル（名称）。言語タグ混在の可能性がある点に注意（必要なら lang フィルタをクエリ側で）
    item["name"] = result["name"]["value"]
    # 備考: 'abstruct' は 'abstract' の綴り誤りと思われるが、挙動を変えず原文に合わせる
    item["abstruct"] = result["abstract"]["value"].replace("\n", "")
    items.append(item)

# 抽出結果から先頭5行を表示（可視確認）
# - ensure_ascii=False により日本語を \uXXXX でなくそのまま出力
# - 出力が長くなる場合は先頭 N 件だけをダンプ（本例は5件）
for item in items[:5]:
    print(json.dumps(item, indent=2, ensure_ascii=False))

In [3]:
# リスト2.1.27
# SPARQL 検索例2
# 手塚治虫文化賞の受賞作家と作品名の一覧を取得
# ------------------------------------------------------------------------------------------------
# 目的:
#   - 日本語 DBpedia の SPARQL エンドポイントに対し、手塚治虫文化賞 (dbp:手塚治虫文化賞) を
#     受賞した「漫画家（ComicsCreator）」と、その代表作（notableWork）を問い合わせ、作家名と作品名を列挙する。
#
# 理論メモ（変更は加えず説明のみ）:
#   - RDF/SPARQLの基本:
#       * RDF は (主語, 述語, 目的語) の三つ組で知識を表現する。SPARQL はそのパターンマッチを行う言語。
#   - 語彙と接頭辞（prefix）:
#       * PREFIX dbp:      <http://ja.dbpedia.org/resource/>   … 日本語 DBpedia のリソース（実体）URI
#       * PREFIX dbp-owl:  <http://dbpedia.org/ontology/>       … DBpedia のオントロジー（概念・性質）URI
#       * PREFIX rdfs:     <http://www.w3.org/2000/01/rdf-schema#> … rdfs:label などの語彙
#       ※ 本クエリでは "dbp-owl" という名前で DBpedia オントロジー (dbo) を参照している（別名のようなもの）。
#   - クエリの意味論（行はそのまま、意味だけ解説）:
#       * ?creator a dbp-owl:ComicsCreator
#           - 変数 ?creator が「漫画家」というクラス（型）に属する個体であることを指定（rdf:type の略記 "a"）。
#       * ?creator dbp-owl:award dbp:手塚治虫文化賞
#           - その個体が手塚治虫文化賞を受賞していることを指定。
#       * ?creator dbp-owl:notableWork ?comic
#           - その個体の代表作（notableWork）として ?comic（作品リソース）を取得。
#       * rdfs:label ?creatorName / ?comics
#           - 作家リソース/作品リソースの人間可読名（ラベル）を取得。
#   - ラベルの言語タグ:
#       * rdfs:label は多言語（@ja, @en など）で存在しうる。ここでは FILTER を入れていないため、
#         実行環境やデータ状況により複数言語が混ざる可能性がある（実務では FILTER(lang(?creatorName)='ja') 等を付与することが多い）。
#   - 結果の決定性:
#       * ORDER BY を付けない SPARQL は結果順序の保証がない。差分の安定性が必要なら ORDER を追加する（本コードでは変更しない）。
#   - エンドポイント/ネットワーク:
#       * HTTP/TCP 到達性やタイムアウトに依存。大きな結果集合では timeout, LIMIT/OFFSET を検討（ここでは最小例のまま）。
# ------------------------------------------------------------------------------------------------

from SPARQLWrapper import SPARQLWrapper

# 日本語 DBpedia の SPARQL エンドポイントへ接続
# returnFormat='json' を指定すると、結果は SPARQL JSON 形式で返る
sparql = SPARQLWrapper(endpoint="http://ja.dbpedia.org/sparql", returnFormat="json")

# 受賞作家 (?creatorName) と代表作 (?comics) を取得するクエリを設定
# - PREFIX で語彙の名前空間を宣言
# - 述語パスの意図:
#     a dbp-owl:ComicsCreator        → 漫画家であること
#     dbp-owl:award 手塚治虫文化賞   → 受賞者であること
#     dbp-owl:notableWork ?comic     → 代表作へのリンク
#     rdfs:label                     → 人間可読なラベル文字列
sparql.setQuery(
    """
PREFIX dbp:      <http://ja.dbpedia.org/resource/> 
PREFIX dbp-owl: <http://dbpedia.org/ontology/>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?creatorName ?comics
{
?creator a  dbp-owl:ComicsCreator;
dbp-owl:award dbp:手塚治虫文化賞;
dbp-owl:notableWork ?comic;
rdfs:label ?creatorName.
?comic rdfs:label ?comics .
}
"""
)

# クエリを送信し、結果を JSON に変換
# - 返り値の構造: { "head": {"vars":[...]}, "results": {"bindings":[ {...}, ... ] } }
# - 各 binding は { "変数名": {"type":"literal/uri", "value":"...", "xml:lang":"ja"(あれば)} } の形
results = sparql.query().convert()

# 検索結果から作家名(creatorName)と作品名(comics)を抽出
# - 本サンプルは結果を Python の辞書リストへ射影し、後続の表示に用いる
# - 言語タグ混在の可能性は考慮せず、そのまま value を採用（最小例のため）
import json

items = []
for result in results["results"]["bindings"]:
    item = {}
    item["creatorName"] = result["creatorName"]["value"]  # 作家名（rdfs:label）
    item["comics"] = result["comics"]["value"]  # 作品名（rdfs:label）
    items.append(item)

# 抽出結果から先頭5行のみを整形表示
# - ensure_ascii=False: 日本語を \uXXXX ではなくそのまま出力
# - 収集が大きくなる場合は LIMIT をクエリ側に追加するのがサーバ負荷/可観測性の面で無難
for item in items[:5]:
    print(json.dumps(item, indent=2, ensure_ascii=False))